<a href="https://colab.research.google.com/github/OliBomby/Mapperatorinator/blob/main/colab/mapperatorinator_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beatmap Generation with Mapperatorinator

This notebook is an interactive demo of an osu! beatmap generation model created by OliBomby. This model is capable of generating hit objects, hitsounds, timing, kiai times, and SVs for all 4 gamemodes. You can upload a beatmap to give to the model as additional context or remap parts of the beatmap.

### Instructions for running:

* Read and accept the rules regarding using this tool
* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each cell to execute the cell
* __Upload Audio__, choose a .mp3 or .ogg file from your computer
* __Upload Beatmap__, optionally choose a .osu file from your computer
* __Configure__, choose your generation parameters to control the style of the generated beatmap
* Generate the beatmap using the __Generate Beatmap__ cell (it may take a few minutes depending on the length of the song)


In [ ]:
#@title Setup Environment { display-mode: "form" }
#@markdown ### Use this tool responsibly. Always disclose the use of AI in your beatmaps.
i_accept_the_rules = False # @param {type:"boolean"}

assert i_accept_the_rules, "Read and accept the rules first!"

!git clone https://github.com/Tiger14n/Mapperatorinator2
%cd Mapperatorinator2

!pip install -r requirements.txt


In [ ]:
%cd /content/Mapperatorinator2


from google.colab import files

import os
import glob
from hydra import compose, initialize
from osuT5.osuT5.event import ContextType
from inference import main

output_path = "output"
input_audio = ""
input_beatmap = ""

#@title Upload Audio { display-mode: "form" }
#@markdown This is the song to generate a beatmap for. Please upload a .mp3 or .ogg file.

def upload_audio():
    data = list(files.upload().keys())
    if len(data) > 1:
        print('Multiple files uploaded; using only one.')
    file = data[0]
    if not file.endswith('.mp3') and not file.endswith('.ogg'):
        print('Invalid file format. Please upload a .mp3 or .ogg file.')
        return ""
    return data[0]

input_audio = upload_audio()

In [ ]:
#@title Configure and Generate Beatmap { display-mode: "form" }

#@markdown #### You can input -1 to leave the value unknown.
#@markdown ---
#@markdown This is model only supports std
gamemode = "standard" # @param ["standard"]
#@markdown This is the Star Rating you want your beatmap to be. It might deviate from this number depending on the song intensity and other configuration.
difficulty = 7 # @param {type:"number"}
#@markdown This is the user ID of the ranked mapper to imitate for mapping style. You can find this in the URL of the mapper's profile.
mapper_id = -1 # @param {type:"integer"}
#@markdown This is the global slider velocity multiplier for the beatmap.
slider_multiplier = 1.4 # @param {type:"slider", min:0.4, max:3.6, step:0.1}
#@markdown This is the circle size (CS) of the beatmap.
circle_size = 3 # @param {type:"number"}

#@markdown ---
#@markdown If true, the generated beatmap will be exported as a .osz file. Otherwise, it will be exported as a .osu file.
export_osz = False # @param {type:"boolean"}

#@markdown This is the temperature of the sampling. A lower temperature will make the model more conservative and map everything like a hard diff. I only recommend lowering this when using `add_to_beatmap` and generating small sections.
temperature = 0.9 # @param {type:"slider", min:0, max:1, step:0.01}
#@markdown This is the random seed. Change this to sample a different beatmap with the same settings.
seed = -1 # @param {type:"integer"}
#@markdown ---


# Get actual parameters
a_gamemode = ["standard", "taiko", "catch the beat", "mania"].index(gamemode)
a_difficulty = None if difficulty == -1 else difficulty
a_mapper_id = None if mapper_id == -1 else mapper_id
a_circle_size = None if circle_size == -1 else circle_size
a_seed = None if seed == -1 else seed

# Validate stuff

assert os.path.exists(input_audio), "Please upload an audio file."

# Create config
with initialize(version_base="1.1", config_path="configs"):
    conf = compose(config_name="inference_v30")

# Do inference
conf.audio_path = input_audio
conf.output_path = output_path
conf.beatmap_path = input_beatmap
conf.gamemode = a_gamemode
conf.difficulty = a_difficulty
conf.mapper_id = a_mapper_id
conf.slider_multiplier = slider_multiplier
conf.circle_size = a_circle_size
conf.export_osz = export_osz
conf.temperature = temperature
conf.seed = a_seed

_, result_path, osz_path = main(conf)

if osz_path is not None:
    result_path = osz_path

if conf.add_to_beatmap:
    files.download(result_path)
else:
    files.download(result_path)
